In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [60]:
##to scan photos

cap = cv2.VideoCapture(0 , cv2.CAP_DSHOW)
count = 0
while True :
    status , photo = cap.read()
    face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    face_cor = face_model.detectMultiScale(photo)
    if len(face_cor) != 0:
        count = count +1
        x , y , h , w = face_cor[0,0] , face_cor[0,1] , face_cor[0,2] , face_cor[0,3]
        crop_photo = photo[y:y+h , x:x+w]
        crop_resize = cv2.resize(crop_photo,(224,224))
        file_name_path = 'C://Users//m27sa//Desktop//MLOps//Deep Learning//Faces//validation//Shobha//image' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, crop_resize)
        cv2.putText(crop_resize , str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper' , crop_resize)
    if cv2.waitKey(1) == 13 or count == 1000 :
        break
cv2.destroyAllWindows()
cap.release()

In [61]:
from keras.applications import VGG16
img_rows = 224
img_cols = 224
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [62]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [63]:
for layer in model.layers:
    layer.trainable = False

In [64]:
def addTopModel(bottom_model, num_classes, D=256):
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [65]:
model.input

<tf.Tensor 'input_4:0' shape=(None, 224, 224, 3) dtype=float32>

In [66]:
model.layers

In [67]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 4

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [68]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Faces/train/'
validation_data_dir = 'Faces/validation/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

Found 802 images belonging to 4 classes.
Found 303 images belonging to 4 classes.


In [69]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("faces.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 3
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator)

Epoch 1/4
802/802 [==============================] - 967s 1s/step - loss: 1.7911 - accuracy: 0.7631 - val_loss: 4.1722e-05 - val_accuracy: 0.9835

Epoch 00001: val_loss improved from inf to 0.00004, saving model to faces.h5
Epoch 2/4
802/802 [==============================] - 964s 1s/step - loss: 0.2865 - accuracy: 0.9377 - val_loss: 0.0000e+00 - val_accuracy: 0.9175

Epoch 00002: val_loss improved from 0.00004 to 0.00000, saving model to faces.h5
Epoch 3/4
802/802 [==============================] - 964s 1s/step - loss: 0.2505 - accuracy: 0.9464 - val_loss: 0.0000e+00 - val_accuracy: 0.9901

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/4
802/802 [==============================] - 962s 1s/step - loss: 0.2084 - accuracy: 0.9688 - val_loss: 0.0000e+00 - val_accuracy: 0.7855

Epoch 00004: val_loss did not improve from 0.00000


In [70]:
modelnew.save("faces.h5")

In [1]:
from keras.models import load_model
model = load_model('faces.h5')

Using TensorFlow backend.


In [27]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0 , cv2.CAP_DSHOW)
while True :
    status , photo = cap.read()
    face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    face_cor = face_model.detectMultiScale(photo)
    if len(face_cor) != 0:
        x , y , h , w = face_cor[0]
        cv2.rectangle(photo , (x,y) , (x+w,y+h) , (0,255,0) , 2 )
        crop_photo = photo[y:y+h , x:x+w]
        crop_resize = cv2.resize(crop_photo,(224,224))
        face = np.expand_dims(crop_resize,axis=0)
        prediction = model.predict(face)
        print(prediction)
        #cv2.putText(photo , str(prediction), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        
        if prediction[0][0] == max(prediction[0]) :
            cv2.putText(photo, "Hey ASHISH", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            #cv2.imshow('Face Recognition', photo )
            
        elif prediction[0][1] == max(prediction[0]) :
            cv2.putText(photo, "Hey POORVI", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            #cv2.imshow('Face Recognition', photo )
            
        elif prediction[0][2] == max(prediction[0]) :
            cv2.putText(photo, "Hey PUSHPA", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            #cv2.imshow('Face Recognition', photo )
            
        elif prediction[0][3] == max(prediction[0]) :
            cv2.putText(photo, "Hey SHOBHA", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            #cv2.imshow('Face Recognition', photo )
            
    cv2.imshow('face' , photo)
    if cv2.waitKey(1) == 13 :
        break
cap.release()
cv2.destroyAllWindows()

[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[0. 0. 0. 1.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]
[[0. 0. 0. 1.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]
[[1. 0. 0. 0.]]


In [11]:
cap.release()

In [12]:
train_generator.class_indices

NameError: name 'train_generator' is not defined

In [13]:
a = [[1,3,4,5]]


In [15]:
max(a[0])

5

In [20]:
if a[0][3] == max(a[0]):
    print("ashish")

ashish


In [21]:
x,y,w,z = a[0]


In [22]:
x

1

In [23]:
y

3

In [24]:
w

4

In [25]:
z

5